In [15]:
import numpy as np
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dropout, BatchNormalization
from keras.layers import Dense
from keras.callbacks import EarlyStopping
from keras.callbacks import LearningRateScheduler
from keras.initializers import RandomNormal
from imblearn.over_sampling import SMOTE
from keras.optimizers import Adam

In [16]:
dataset = pd.read_csv("Churn_Modelling.csv")
X = dataset.iloc[:, 3:-1].values 
y = dataset.iloc[:, -1].values   

In [17]:
geography_encoder = LabelEncoder()
geography_encoder.fit(dataset['Geography']) 
gender_encoder = LabelEncoder()
gender_encoder.fit(dataset['Gender'])

LabelEncoder()

In [18]:
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(handle_unknown='ignore'), [1, 2])], remainder='passthrough')  
X = np.array(ct.fit_transform(X))

In [19]:
sc = StandardScaler()
X = sc.fit_transform(X) 

In [20]:
#X, y = SMOTE(random_state=0).fit_resample(X, y)

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [9]:
ann_model = Sequential()

ann_model.add(Dense(units=12, kernel_initializer=RandomNormal(mean=0.0, stddev=0.05, seed=None), activation='relu'))
ann_model.add(BatchNormalization())
ann_model.add(Dropout(0.1))  

ann_model.add(Dense(units=12, kernel_initializer=RandomNormal(mean=0.0, stddev=0.05, seed=None), activation='relu'))
ann_model.add(BatchNormalization())
ann_model.add(Dropout(0.1))  

ann_model.add(Dense(units=1, activation='sigmoid'))
ann_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
ann_model.build(input_shape=(None, X_train.shape[1]))

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)

def lr_schedule(epoch):
    lr = 0.01
    if epoch > 75:
        lr = 0.001
    elif epoch > 50:
       lr = 0.005
    print('Learning rate: ', lr)
    return lr
lr_scheduler = LearningRateScheduler(lr_schedule)

ann_model.fit(X_train, y_train, batch_size=32, epochs=200, callbacks=[es, lr_scheduler], validation_split=0.2)

Learning rate:  0.01
Epoch 1/200
200/200 [==============================] - 2s 4ms/step - loss: 0.4479 - accuracy: 0.7958 - val_loss: 0.4479 - val_accuracy: 0.7969 - lr: 0.0100
Learning rate:  0.01
Epoch 2/200
200/200 [==============================] - 0s 2ms/step - loss: 0.3968 - accuracy: 0.8284 - val_loss: 0.3821 - val_accuracy: 0.8425 - lr: 0.0100
Learning rate:  0.01
Epoch 3/200
200/200 [==============================] - 0s 2ms/step - loss: 0.3808 - accuracy: 0.8392 - val_loss: 0.3671 - val_accuracy: 0.8519 - lr: 0.0100
Learning rate:  0.01
Epoch 4/200
200/200 [==============================] - 0s 2ms/step - loss: 0.3736 - accuracy: 0.8427 - val_loss: 0.3646 - val_accuracy: 0.8481 - lr: 0.0100
Learning rate:  0.01
Epoch 5/200
200/200 [==============================] - 0s 2ms/step - loss: 0.3716 - accuracy: 0.8423 - val_loss: 0.3595 - val_accuracy: 0.8587 - lr: 0.0100
Learning rate:  0.01
Epoch 6/200
200/200 [==============================] - 0s 2ms/step - loss: 0.3747 - accuracy: 

In [10]:
new_observation = np.array([[600, 'Spain', 'Female', 40, 3, 60000, 2, 1, 1, 50000]]) 

new_observation[:, 1] = geography_encoder.transform(new_observation[:, 1].reshape(-1, 1)) 
new_observation[:, 2] = gender_encoder.transform(new_observation[:, 2].reshape(-1, 1)) 

new_observation = ct.transform(new_observation)

new_observation = sc.transform(new_observation)


C:\Users\ahmed\anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:132: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)
C:\Users\ahmed\anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:132: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


In [11]:
new_prediction = ann_model.predict(new_observation)[0]
probability = new_prediction
new_prediction = (new_prediction > 0.5)


prediction_result = "Will Leave" if new_prediction else "Will Stay"
print(f"Prediction: {prediction_result}")
print(f"Probability of leaving: {probability}") 


1/1 [==============================] - 0s 140ms/step
Prediction: Will Stay
Probability of leaving: [0.05230134]


In [12]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

y_pred = ann_model.predict(X_test)
y_pred = (y_pred > 0.5)

cm = confusion_matrix(y_test, y_pred)

accuracy = accuracy_score(y_test, y_pred)

precision = precision_score(y_test, y_pred)

recall = recall_score(y_test, y_pred)

f1 = f1_score(y_test, y_pred)

auc_roc = roc_auc_score(y_test, y_pred)

print(f"Confusion Matrix:\n{cm}")
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")
print(f"AUC-ROC: {auc_roc}")


63/63 [==============================] - 0s 968us/step
Confusion Matrix:
[[1538   57]
 [ 216  189]]
Accuracy: 0.8635
Precision: 0.7682926829268293
Recall: 0.4666666666666667
F1 Score: 0.5806451612903226
AUC-ROC: 0.7154649947753396


In [14]:
import joblib

ann_model.save('ann_model.h5')

joblib.dump(geography_encoder, 'geography_encoder.pkl')
joblib.dump(gender_encoder, 'gender_encoder.pkl')
joblib.dump(ct, 'column_transformer.pkl')
joblib.dump(sc, 'standard_scaler.pkl')


['standard_scaler.pkl']